In [90]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)


In [91]:
PATH_TO_PRODUCTS = '../data/raw_data/Product Catalogue/'
PATH_TO_RATINGS = '../data/raw_data/Rating and Reviews/'
NB_ROWS_PER_BATCH = 200000

In [92]:
for filename in os.listdir(PATH_TO_PRODUCTS):
    print(filename)

Cosmetics_Product_20190831.csv
Cosmetics_Product_20190930.csv
Cosmetics_Product_20191031.csv
cosmetics_product_2019Q4.csv
Cosmetics_Product_20200116.csv
Global Product Catalogs - ELC ST Fix
Historic Product Catalog - ELC ST Fix.b
Historic Product Catalogs - ELC ST Fix
SkinCare_New_Product_20191031.zip
SkinCare_Product_20190630_ST MV2SV.csv
SkinCare_Product_20190930.csv
SkinCare_Product_20190930_Update.csv
SkinCare_Product_20191031.csv
skincare_product_20191231.csv
skincare_product_2019Q3.csv
skincare_product_2019Q3_final.csv
SkinCare_Product_20200116.csv


In [93]:
for filename in os.listdir(PATH_TO_RATINGS):
    print(filename)

Cosmetics_Reviews_20190630.csv
Cosmetics_Reviews_20190831.csv
Cosmetics_Reviews_20190930.csv
Cosmetics_Reviews_20191031.csv
cosmetics_reviews_20191130_final.csv
cosmetics_reviews_20191231.csv
cosmetics_reviews_20200101-20200131.csv
cosmetics_reviews_20200229.csv
deleted
pwds
Skin Care_Reviews_20190930.csv
skincare_reviews_20150201-20200131.csv
SkinCare_Reviews_20190630_ST MV2SV.csv
SkinCare_Reviews_20190831.csv
SkinCare_Reviews_20191031.csv
skincare_reviews_20191130.csv
skincare_reviews_20191231.csv
skincare_reviews_20200229.csv
template


In [94]:
product_template = pd.read_csv('../data/raw_data/Rating and Reviews/template/sProduct.csv')
product_template.columns

Index(['ImportID', 'FileSourceID', 'Product_Id', 'Source_Product_Identifier',
       'Product', 'Description', 'Channel', 'Brand', 'Feature', 'Benefit',
       'Ingredient', 'Additional_Ingredients', 'Product_Form',
       'ELC_Solution_Type', 'Rating', 'Number_of_Reviews', 'Geography',
       'Collection_Date', 'Normalized_Product_Title', 'ProductCluster_Id',
       'Finish', 'Looks', 'Other', 'Trends', 'Source_file_loaded',
       'Load_Date'],
      dtype='object')

In [95]:
ratings_template = pd.read_csv('../data/raw_data/Rating and Reviews/template/sReviewTheme.csv')
ratings_template.columns

Index(['ImportID', 'FileSourceID', 'OnlinePost_ID',
       'Source_Product_Identifier', 'OnlineStatement_ID', 'Review_Date',
       'Title', 'Description', 'Geography', 'Channel', 'Product_ID', 'Rating',
       'Sentiment', 'Feature', 'Benefit', 'Ingredient',
       'Additional_Ingredients', 'Product_Form', 'ELC_Solution_Type',
       'Skin_Condition', 'Packaging', 'Skin_Type', 'Treatment_Area',
       'Use_Case', 'Syndication_Source', 'Best_For', 'Verified_Buyer', '_From',
       'Recommended', 'Verified_Reviewer', 'Eye_Color', 'Hair_Color',
       'Skin_Tone', 'Gender', 'I_shop_at_macys_com', 'Make_up_Style',
       'Purchase_Location', 'Cons', 'Pros', 'Describe_Yourself',
       'Reviewer_Skin_Type', 'Age', 'Source_File_Loaded', 'Load_Date', 'Looks',
       'Other', 'Trends'],
      dtype='object')

In [96]:
ratings_columns_mapping = {
    'OnlinePost_ID':'OnlinePost_ID',
    'Source Product Identifier':'Source_Product_Identifier',
    'OnlineStatement_ID':'OnlineStatement_ID',
    'Date':'Review_Date', 
    'Title':'Title',
    'Description':'Description',
    'Geography':'Geography',
    'Channel':'Channel',
    'Product_ID':'Product_ID',
    'Rating':'Rating',
    'Sentiment':'Sentiment',
    'Feature':'Feature',
    'Benefit':'Benefit',
    'Ingredient':'Ingredient',
    'Additional Ingredients (no rulebase)':'Additional_Ingredients',
    'Product Form':'Product_Form',
    'ELC Solution Type':'ELC_Solution_Type',
    'Skin Condition':'Skin_Condition',
    'Packaging':'Packaging',
    'Skin Type':'Skin_Type',
    'Treatment Area':'Treatment_Area',
    'Use Case':'Use_Case',
    'Syndication Source':'Syndication_Source',
    'Best For':'Best_For',
    'Verified Buyer':'Verified_Buyer',
    'From':'_From', 
    ' From':'_From',
    'Recommended':'Recommended',
    'Verified Reviewer':'Verified_Reviewer',
    'Eye Color':'Eye_Color',
    'Hair Color':'Hair_Color', 
    'Skin Tone':'Skin_Tone',
    'Gender':'Gender', 
    'I shop at macys.com':'I_shop_at_macys_com',
    'I shop at\xa0macys.com':'I_shop_at_macys_com',
    'Make-up Style':'Make_up_Style',
    'Purchase Location':'Purchase_Location',
    'Cons':'Cons',
    'Pros':'Pros',
    'Describe Yourself':'Describe_Yourself',
    'Reviewer Skin Type':'Reviewer_Skin_Type',
    'Age':'Age'
}

In [97]:
product_columns_mapping = {
    'Product_ID':'Product_Id', 
    'Source Product Identifier':'Source_Product_Identifier',
    'Product':'Product',
    'DescriptioChanneln':'Description',
    'Channel':'Channel',
    'Brand':'Brand',
    'Feature':'Feature',
    'Benefit':'Benefit',
    'Ingredient':'Ingredient',
    'Additional Ingredients (no rulebase)':'Additional_Ingredients',
    'Product Form':'Product_Form',
    'ELC Solution Type':'ELC_Solution_Type',
    'Finish':'Finish',
    'Looks':'Looks',
    'Other':'Other',
    'Trends':'Trends',
    'Rating':'Rating',
    'Number of Reviews':'Number_of_Reviews',
    'Geography':'Geography', 
    'Collection Date':'Collection_Date',
    'Normalized Product Title':'Normalized_Product_Title',
    'ProductCluster_ID':'ProductCluster_Id', 
    'Cluster Size':''
}

In [98]:
def split_data(input_path, 
               rows_per_batch = 150000, 
               output_path = '../data/to_import/',
               ratings_columns=ratings_template.columns.tolist(),
               product_columns=product_template.columns.tolist(),
               ratings_mapping = ratings_columns_mapping,
               product_mapping = product_columns_mapping
              ):
    
    for filename in tqdm(os.listdir(input_path)):
        if filename.endswith('.csv'):
            if filename.lower().find('product')!=-1:
                file_type = 'products'
                template_columns = product_columns
                mapping = product_mapping
                if filename.lower().startswith('cosm'):
                    product_type = 'cosmetics'
                elif filename.lower().startswith('skin'):
                    product_type = 'skincare'   
                
                
            elif filename.lower().find('reviews')!=-1:
                file_type = 'reviews'
                template_columns = ratings_columns
                mapping = ratings_mapping
                if filename.lower().startswith('cosm'):
                    product_type = 'cosmetics'
                elif filename.lower().startswith('skin'):
                    product_type = 'skincare' 
                    
            file_dir = os.path.join(output_path, file_type, product_type, filename.replace('.csv',''))
            if not os.path.exists(file_dir):
                os.makedirs(file_dir)
            print(filename)
            df=pd.read_csv(os.path.join(input_path, filename), low_memory=False)
            df.rename(columns=mapping, inplace=True)
            columns_to_add = [col for col in template_columns if not col in df.columns] 
            columns_to_drop = [col for col in df.columns if not col in template_columns]
            print('Columns to drop : ', columns_to_drop)
            print('Columns to add : ', columns_to_add)
            for col in columns_to_add:
                df[col]=np.nan
            df=df[template_columns]
            nb_files = int(np.ceil(len(df)/rows_per_batch))
            for i in range(nb_files):
                batch = df.iloc[i*rows_per_batch:(i+1)*rows_per_batch,:]
                if nb_files==1:
                    batch.to_csv(os.path.join(file_dir, filename), index=False)
                else:
                    batch.to_csv(os.path.join(file_dir, filename.replace('.csv','_'+str(i+1)+'.csv')), index=False)

In [100]:
split_data(PATH_TO_PRODUCTS, rows_per_batch=NB_ROWS_PER_BATCH)




  0%|                                                                                           | 0/17 [00:00<?, ?it/s]

Cosmetics_Product_20190831.csv
Columns to drop :  ['']
Columns to add :  ['ImportID', 'FileSourceID', 'Source_file_loaded', 'Load_Date']





  6%|████▉                                                                              | 1/17 [00:02<00:35,  2.24s/it]

Cosmetics_Product_20190930.csv
Columns to drop :  ['']
Columns to add :  ['ImportID', 'FileSourceID', 'Source_file_loaded', 'Load_Date']





 12%|█████████▊                                                                         | 2/17 [00:04<00:33,  2.22s/it]

...




 88%|████████████████████████████████████████████████████████████████████████▎         | 15/17 [00:42<00:08,  4.41s/it]

skincare_product_2019Q3_final.csv
Columns to drop :  ['Skin Condition', 'Packaging', 'Skin Type', 'Treatment Area', 'Use Case']
Columns to add :  ['ImportID', 'FileSourceID', 'Finish', 'Looks', 'Other', 'Trends', 'Source_file_loaded', 'Load_Date']





 94%|█████████████████████████████████████████████████████████████████████████████▏    | 16/17 [00:47<00:04,  4.42s/it]

SkinCare_Product_20200116.csv
Columns to drop :  ['Skin Condition', 'Packaging', 'Skin Type', 'Treatment Area', 'Use Case']
Columns to add :  ['ImportID', 'FileSourceID', 'Finish', 'Looks', 'Other', 'Trends', 'Source_file_loaded', 'Load_Date']





100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:51<00:00,  3.03s/it]


In [23]:
split_data(PATH_TO_RATINGS, rows_per_batch=NB_ROWS_PER_BATCH)

In [24]:
['ImportID', 'FileSourceID', '',
       '', '', '',
       '', '', '', '', '', '',
       '', '', '', '',
       '', '', '',
       '', '', '', '',
       '', '', '', '', '',
       '', '', '', '',
       '', '', '', '',
       '', '', '', '',
       '', '', 'Source_File_Loaded', 'Load_Date', 'Looks',
       'Other', 'Trends']

In [ ]:
['ImportID', 'FileSourceID', '', '',
       '', '', '', '', '', '',
       '', '', '',
       '', '', '', '',
       '', '', '',
       '', '', '', '', 'Source_file_loaded',
       'Load_Date']